In [51]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiLabelClassificationModel

In [43]:
pwd

'C:\\Users\\Mateusz\\OneDrive\\Pulpit\\Nauka_vol2\\Informatyka\\ML_and_DL'

In [44]:
df = pd.read_csv('offensive\\train.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [45]:
df['labels'] = list(zip(df.toxic.tolist(), df.severe_toxic.tolist(), df.obscene.tolist(), df.threat.tolist(),  df.insult.tolist(), df.identity_hate.tolist()))
df['text'] = df['comment_text'].apply(lambda x: x.replace('\n', ' '))
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",Explanation Why the edits made under my userna...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",D'aww! He matches this background colour I'm s...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)","Hey man, I'm really not trying to edit war. It..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",""" More I can't make any real suggestions on im..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)","You, sir, are my hero. Any chance you remember..."


In [46]:
sum(df["identity_hate"]==1)

1405

In [48]:
train_df, eval_df = train_test_split(df, test_size=0.2)

In [50]:
#model był trenowany, obecnie wczytujemy go z checkpointa
model = MultiLabelClassificationModel('roberta', 'outputs/checkpoint-2811-epoch-3', num_labels=6, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 128})

In [9]:
#obcinamy set danych, jest za duży jak na ilość naszego czasu do pretrenowania
train_df = train_df[:30000]
eval_df = eval_df[:5000]

In [9]:
model.train_model(train_df, eval_df=eval_df)

  0%|          | 0/30000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/15000 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/15000 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/15000 [00:00<?, ?it/s]

(2811, 0.054602887294200604)

In [ ]:
y_true_global, y_pred_global = 0, 0

def helper(y_true, y_pred):
    global y_true_global, y_pred_global
    y_true_global, y_pred_global = y_true, y_pred
    #jednak tutaj tak wygoodnie to nie zadziała, trzeba mierzyć F1 po każdej z klas
    return sklearn.metrics.f1_score(y_true,y_pred,average="samples")

result, model_outputs, wrong_predictions = model.eval_model(
    eval_df, acc = helper
)

In [11]:
y_true_global

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [13]:
y_pred_global

array([[1.14250183e-03, 6.48498535e-04, 7.55310059e-04, 6.74724579e-04,
        7.91549683e-04, 7.73429871e-04],
       [1.27887726e-03, 5.72681427e-04, 7.58171082e-04, 5.95569611e-04,
        7.55310059e-04, 6.90460205e-04],
       [6.61621094e-01, 1.82437897e-03, 2.69042969e-01, 1.42097473e-03,
        4.16259766e-02, 3.36265564e-03],
       ...,
       [9.55078125e-01, 4.92248535e-02, 9.48242188e-01, 6.12258911e-03,
        3.41064453e-01, 1.41143799e-02],
       [1.48391724e-03, 5.19275665e-04, 7.43865967e-04, 5.56945801e-04,
        7.38143921e-04, 6.64234161e-04],
       [1.83868408e-03, 4.61816788e-04, 8.03947449e-04, 5.01155853e-04,
        7.58171082e-04, 6.07013702e-04]])

In [33]:
best_thresholds_f1 = [0]*6
best_scores_f1 = [0]*6
for step in np.arange(0, 1, 0.01):
    casted_true = np.where(y_pred_global > step, 1, 0)
    for i in range(6): 
        score = sklearn.metrics.f1_score([x[i] for x in y_true_global], [x[i] for x in casted_true])
        if best_scores_f1[i] < score:
            best_thresholds_f1[i] = step
            best_scores_f1[i] = score
print(list(zip(best_scores_f1, best_thresholds_f1)))

[(0.8554095045500506, 0.62), (0.5454545454545455, 0.38), (0.8297101449275363, 0.53), (0.5833333333333334, 0.17), (0.7784679089026915, 0.63), (0.39999999999999997, 0.23)]


In [30]:
best_thresholds_acc = [0]*6
best_scores_acc = [0]*6
for step in np.arange(0, 1, 0.1):
    casted_true = np.where(y_pred_global > step, 1, 0)
    for i in range(6): 
        score = sklearn.metrics.accuracy_score([x[i] for x in y_true_global], [x[i] for x in casted_true])
        if best_scores_acc[i] < score:
            best_thresholds_acc[i] = step
            best_scores_acc[i] = score
print(list(zip(best_scores_acc, best_thresholds_acc)))

[0.9708, 0.9932, 0.982, 0.998, 0.978, 0.9932]


In [26]:
best_thresholds_recall = [0]*6
best_scores_recall = [0]*6
for step in np.arange(0, 1, 0.1):
    casted_true = np.where(y_pred_global > step, 1, 0)
    for i in range(6): 
        score = sklearn.metrics.recall_score([x[i] for x in y_true_global], [x[i] for x in casted_true])
        if best_scores_recall[i] < score:
            best_thresholds_recall[i] = step
            best_scores_recall[i] = score
print(list(zip(best_scores_recall, best_thresholds_recall)))

0 0.8882235528942116
1 0.6
2 0.8973384030418251
3 0.35714285714285715
4 0.8697478991596639
5 0.17142857142857143


In [27]:
best_thresholds_precision = [0]*6
best_scores_precision = [0]*6
for step in np.arange(0, 1, 0.1):
    casted_true = np.where(y_pred_global > step, 1, 0)
    for i in range(6): 
        score = sklearn.metrics.precision_score([x[i] for x in y_true_global], [x[i] for x in casted_true])
        if best_scores_precision[i] < score:
            best_thresholds_precision[i] = step
            best_scores_precision[i] = score
print(list(zip(best_scores_precision, best_thresholds_precision)))

0 0.8061594202898551
1 0.4444444444444444
2 0.7515923566878981
3 0.8333333333333334
4 0.6764705882352942
5 0.35294117647058826


In [24]:
casted_true.nonzero()[0].size/(casted_true.shape[0]*casted_true.shape[1])

0.041633333333333335

In [11]:
result

{'LRAP': 0.9955914999999994, 'eval_loss': 0.041696573441009965}

In [12]:
model_outputs

array([[0.00121117, 0.00059319, 0.00074959, 0.0006237 , 0.00075245,
        0.00072956],
       [0.00121593, 0.00062132, 0.00076437, 0.00064611, 0.00077915,
        0.0007267 ],
       [0.00133038, 0.00056171, 0.00076437, 0.00058365, 0.00074387,
        0.00068808],
       ...,
       [0.00152493, 0.00050545, 0.00076437, 0.00053167, 0.00074959,
        0.00063848],
       [0.00173378, 0.00046921, 0.00084591, 0.00050116, 0.00073195,
        0.0006094 ],
       [0.00135136, 0.00054216, 0.00075531, 0.00056362, 0.00074673,
        0.00066662]])

In [13]:
wrong_predictions

[]